# Zadanie

1. To jest szablon zadania - wypełnij przed wysłaniem
2. Zadanie polega na opracowaniu bazy danych plikowej oraz konfiguracji środowiska (np. serwera mongodb w docker) i przedstawieniu wykonanej bazy.
3. Zaimplementuj w bazie plikowej swoją bazę z pierwszego zadania i wykonaj dla niej następujące zapytania:
 1.  Wykonaj podstawowe zapytania pokazujące wprowadzenie danych (3 x wyszukiwania pojedynczych wierszy).   {ocena 3.0-3.5}
 
 2. Dodaj wyszukiwania grup wierszy (lista wierszy; 3 x przykładowe zapytanie)    {ocena 3.5-4.0}
 3. Dodaj  wyliczania wartości dla zapytania (3 x przykładowe zapytanie wyliczające)    {ocena 4.0-4.5}
 4. Dodaj wyszukiwania z agregacją wyników (3 x przykładowe agregacje).   {ocena 4.5-5.0}
 5. Wpisz w ostatniej komórce notatnika swoje spostrzeżenia na temat: "Jak wypada porównanie bazy plikowej w odniesieniu do bazy relacyjnej na przykładzie mojej bazy danych z laboratorium?"


Opis:
-  Wykonaj bazę klucz plikową na podstawie swojego wcześniejszego (zad 1) modelu danych. Warto skorzystać z przygotowanej wcześniej bazy danych. Zastanów sie które rekordy z tamtej bazy danych warto wpisać we dokumentach, coś co wcześniej trzeba było łączyć poleceniem JOIN można teraz wpisać wspólnie w jednym dokuemncie. 
- W zapytaniach użyj podstawowych mechanizmów MongoDB: użyj sortowania, projekcji, poleceń limit i distinct.
- Wykonaj przynajmniej 12 zapytań do bazy (4 x 3 przykłady zapytań [A, B, C, D])

Polecenia:
 - oprogramuj zadanie,
 - prześlij wypełniony notatnik do naszego kursu online,
 - przedstaw działające zadanie na zajęciach laboratoryjnych,
 
- Nazwa pliku notatnika: zad4_nazwisko_imie.ipynb 

## Opis rozwiązania (wypełnia student)

1. Autor: 
 - Dominik Boguszewski
2. temat bazy danych?
 1. Weterynaria


In [ ]:
from pymongo import MongoClient
import pandas as pd
import cx_Oracle
import numpy as np

In [ ]:
try:
    cx_Oracle.init_oracle_client(lib_dir="D:\\Studia\\mag_sem1\\bazy_danych\\laby\\instantclient_21_9")
    mongo = MongoClient("localhost", 50001)
    dsn_tns = cx_Oracle.makedsn('217.173.198.136', '1521', service_name='whdb')
    con = cx_Oracle.connect(user='s98784', password='s98784', dsn=dsn_tns)
    print(f"Connection provided. Connection version: {con.version}")
    print(mongo.list_database_names(), "\n")

except cx_Oracle.DatabaseError as e:
    print("There is a problem with Oracle: ", e)
    mongo.close()
    if con:
        con.close()

In [ ]:
# utworzenie tabel w mongo
cx_lekarz = pd.read_sql("select * from Lekarz", con=con)
cx_urlop = pd.read_sql("select * from Urlop", con=con)
cx_wizyta = pd.read_sql("select * from Wizyta", con=con)
cx_pacjent = pd.read_sql("select * from Pacjent", con=con)
cx_chirurgia = pd.read_sql("select * from Chirurgia", con=con)
cx_diagnostyka = pd.read_sql("select * from Diagnostyka", con=con)

db = mongo["weterynaria"]
lekarz = db["lekarz"]
urlop = db["urlop"]
wizyta = db["wizyta"]
pacjent = db["pacjent"]
chirurgia = db["chirurgia"]
diagnostyka = db["diagnostyka"]

In [ ]:
#  Wykonaj podstawowe zapytania pokazujące wprowadzenie danych (3 x wyszukiwania pojedynczych wierszy).   {ocena 3.0-3.5}
data = cx_lekarz.to_dict('records')
ids = lekarz.insert_many(data)
# for l in lekarz.find(): print(l)

data = cx_urlop.to_dict('records')
ids = urlop.insert_many(data)
# for l in urlop.find(): print(l)

data = cx_wizyta.to_dict('records')
ids = wizyta.insert_many(data)
# for l in wizyta.find(): print(l)

data = cx_pacjent.to_dict('records')
ids = pacjent.insert_many(data)
# for l in pacjent.find(): print(l)

data = cx_chirurgia.to_dict('records')
ids = chirurgia.insert_many(data)
# for l in chirurgia.find(): print(l)

data = cx_diagnostyka.to_dict('records')
ids = diagnostyka.insert_many(data)
# for l in diagnostyka.find(): print(l)


In [ ]:
# Dodaj wyszukiwania grup wierszy (lista wierszy; 3 x przykładowe zapytanie)    {ocena 3.5-4.0}
lek = lekarz.find(filter={"WYPLATA": {"$gt": 5000, "$lt": 6000}})
for i in lek:
    print(f'\nImie: {i["IMIE"]}\nNazwisko: {i["NAZWISKO"]}\nWypłata: {i["WYPLATA"]}')

pac = pacjent.find(filter={"IMIE": "Thayne"})
for i in pac:
    print(f'\nImie: {i["IMIE"]}\nNazwisko: {i["NAZWISKO"]}\nAdres: {i["ADRES"]}')

wiz = wizyta.find(filter={"CENA": {"$gt": 500, "$lt": 1000}})
for i in wiz:
    print(f'\nData: {i["DATA"]}\nId pacjenta: {i["PACJENT_ID"]}\nCena: {i["CENA"]}')

In [ ]:
# Dodaj  wyliczania wartości dla zapytania (3 x przykładowe zapytanie wyliczające)    {ocena 4.0-4.5}


In [ ]:
# Dodaj wyszukiwania z agregacją wyników (3 x przykładowe agregacje).   {ocena 4.5-5.0}
pipeline = [
    {
        '$lookup': {
            'from': 'wizyta',
            'localField': 'ID',
            'foreignField': 'LEKARZ_ID',
            'as': 'wizyty'
        }
    },
    {
        '$project': {
            'imie': 1,
            'nazwisko': 1,
            'liczba_wizyt': {'$size': '$wizyty'}
        }
    }
]

results = lekarz.aggregate(pipeline)

for result in results:
    print(result)

print()

pipeline = [
    {
        '$lookup': {
            'from': 'urlop',
            'localField': 'ID',
            'foreignField': 'LEKARZ_ID',
            'as': 'urlopy'
        }
    },
    {
        '$project': {
            'imie': 1,
            'nazwisko': 1,
            'liczba_urlopow': {'$size': '$urlopy'}
        }
    }
]

results = lekarz.aggregate(pipeline)

for result in results:
    print(result)

print()

pipeline = [
        {
            '$sort': {
                'WYPLATA': -1
            }
        },  
        {
            '$limit': 1
        },  
        {
            '$project': {
                'imie': 1,
                'nazwisko': 1,
                'wyplata': {'$max': '$WYPLATA'}
            }
        } 
    ]

results = lekarz.aggregate(pipeline)

for result in results:
    print(f"Największą wypłatę ma: {result}")

In [ ]:
# Wpisz w ostatniej komórce notatnika swoje spostrzeżenia na temat: 
# "Jak wypada porównanie bazy plikowej w odniesieniu do bazy relacyjnej na przykładzie mojej bazy danych z laboratorium?"

# Porównanie wypada bardzo dobrze, ponieważ gdy raz się zaimportuje bazę danych do mongo,
# to praktycznie pomijany jest czas połączenia z serwerem. W połączeniu oraclowskim czasami czas oczekiwania na samo 
# uzyskanie połączenia to w moim przypadku było od 30 sek do minuty. Lecz w przypadku mongo wszystkie operacje były wykonane
# po zaledwie kilku sekundach.
